In [ ]:
import gym
import numpy as np
import math
from collections import deque
import time
import matplotlib
import matplotlib.pyplot as plt

class MountaincarQAgent():
    def __init__(self, buckets=(12, 12), num_episodes=500000, min_epsilon=0.01, discount=0.99, decay=100, force=True):
        self.buckets = buckets
        self.num_episodes = num_episodes
        self.min_epsilon = min_epsilon
        self.discount = discount
        self.decay = decay

        self.env = gym.make('MountainCar-v0')
        self.upper_bounds = [self.env.observation_space.high[0], self.env.observation_space.high[1]]
        self.lower_bounds = [self.env.observation_space.low[0], self.env.observation_space.low[1]]
        
        ## Concatination of tuples to get shape (12,12,3) for buckets=(12, 12)
        self.Q_table = np.zeros(self.buckets + (self.env.action_space.n,))        
        
        self.learning_rate = 0.008
        
        self.threshold = self.env.spec.reward_threshold
        print('threshold: ', self.threshold)

    def discretize_state(self, obs):
        discretized = list()
        for i in range(len(obs)):
            scaling = (obs[i] + abs(self.lower_bounds[i])) / (self.upper_bounds[i] - self.lower_bounds[i])
            new_obs = int(round((self.buckets[i] - 1) * scaling))
            new_obs = min(self.buckets[i] - 1, max(0, new_obs))
            discretized.append(new_obs)
        return tuple(discretized)

    
    def choose_action(self, state):
        if (np.random.random() < self.epsilon):
            return self.env.action_space.sample() 
        else:
            return np.argmax(self.Q_table[state])

    def update_q(self, state, action, reward, new_state):
        self.Q_table[state][action] += \
           self.learning_rate * (reward + self.discount * np.max(self.Q_table[new_state]) - self.Q_table[state][action])

    def get_epsilon(self, t):
        return max(self.min_epsilon, min(1., 1. - math.log10((t + 1) / self.decay)))

    
    def train(self):
        scores_deque = deque(maxlen=100)
        scores_array = []
        avg_scores_array = []  
        print_every = 400
        time_start = time.time()
        
        for i_episode in range(self.num_episodes):
            current_state = self.discretize_state(self.env.reset())

            self.epsilon = self.get_epsilon(i_episode)
            done = False
            
            episode_reward = 0
            time_step = 0
            
            while not done:
                action = self.choose_action(current_state)
                obs, reward, done, _ = self.env.step(action)   
                new_state = self.discretize_state(obs)
                self.update_q(current_state, action, reward, new_state)
                current_state = new_state
                time_step += 1
                episode_reward += reward
                
            scores_deque.append(episode_reward)
            scores_array.append(episode_reward)
            
            avg_score = np.mean(scores_deque)
            avg_scores_array.append(avg_score)
            
            s = (int)(time.time() - time_start)
            
            if i_episode % print_every == 0 and i_episode > 0:                
                print('Episode: {}, Timesteps:  {}, Score: {:5},  Avg.Score: {:.2f}, eps-greedy: {:5.2f}, Time: {:02}:{:02}:{:02}'.\
                    format(i_episode, time_step, episode_reward, avg_score, self.epsilon, s//3600, s%3600//60, s%60))    
                
            if avg_score >= self.threshold: 
                print('\n Environment solved in {:d} episodes!\tAverage Score: {:.2f}'. \
                    format(i_episode, np.mean(scores_deque)))
                break                                

        print('Finished training!')
        
        return scores_array, avg_scores_array

    def run(self):
        self.env = gym.wrappers.Monitor(self.env,'Mountaincar', force=True)
        t = 0
        done = False
        current_state = self.discretize_state(self.env.reset())
        while not done:
                self.env.render()
                t = t+1
                action = self.choose_action(current_state)
                obs, reward, done, _ = self.env.step(action)
                new_state = self.discretize_state(obs)
                current_state = new_state
            
        return t
    
    def close(self):
        self.env.close()




In [ ]:

agent = MountaincarQAgent()
scores, avg_scores = agent.train()

/usr/local/lib/python3.7/dist-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


threshold:  -110.0
Episode: 400, Timesteps:  200, Score: -200.0,  Avg.Score: -200.00, eps-greedy:  0.40, Time: 00:00:15
Episode: 800, Timesteps:  200, Score: -200.0,  Avg.Score: -200.00, eps-greedy:  0.10, Time: 00:00:28
Episode: 1200, Timesteps:  200, Score: -200.0,  Avg.Score: -199.81, eps-greedy:  0.01, Time: 00:00:46
Episode: 1600, Timesteps:  200, Score: -200.0,  Avg.Score: -198.88, eps-greedy:  0.01, Time: 00:01:09
Episode: 2000, Timesteps:  200, Score: -200.0,  Avg.Score: -199.06, eps-greedy:  0.01, Time: 00:01:28
Episode: 2400, Timesteps:  200, Score: -200.0,  Avg.Score: -196.06, eps-greedy:  0.01, Time: 00:01:43
Episode: 2800, Timesteps:  200, Score: -200.0,  Avg.Score: -184.75, eps-greedy:  0.01, Time: 00:01:57
Episode: 3200, Timesteps:  200, Score: -200.0,  Avg.Score: -189.34, eps-greedy:  0.01, Time: 00:02:12
Episode: 3600, Timesteps:  200, Score: -200.0,  Avg.Score: -199.12, eps-greedy:  0.01, Time: 00:02:27
Episode: 4000, Timesteps:  200, Score: -200.0,  Avg.Score: -190.3

In [ ]:
%matplotlib inline

print('length of scores: ', len(scores), ', len of avg_scores: ', len(avg_scores))

fig = plt.figure(figsize=(15,3))
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores, label="Score")
plt.plot(np.arange(1, len(avg_scores)+1), avg_scores, label="Avg on 100 episodes")
plt.legend(bbox_to_anchor=(1.05, 1)) 
plt.ylabel('Score')
plt.xlabel('Episodes #')
plt.show()

In [ ]:
t = agent.run()
print("Time", t)

In [ ]:
agent.close()